### combine latex


In [4]:
import requests
import urllib.request
from zipfile import ZipFile
import os
from pathlib import Path
import tarfile

# url = "https://arxiv.org/abs/2204.04998"
url = "https://arxiv.org/abs/2202.10855"
latex_source = url.replace("abs", "e-print")
zip_name = latex_source.split("e-print/")[1]

folder_path = f"./verbalist/datasets/openreview/{zip_name}"
save_path = f"{folder_path}.zip"

if not os.path.isfile(save_path):
    urllib.request.urlretrieve(latex_source, save_path)
    
with tarfile.open(save_path) as zip_obj:
    Path(folder_path).mkdir(exist_ok=True)
    zip_obj.extractall(folder_path)